In [1]:
# syft absolute
import syft as sy
from syft import ActionObject

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=9ce7d842a3034d58a2048342a3c2502b in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=572e2de652cf41a4a23df57a3138a239 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.82it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.32it/s]

Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/3973756a73b44ff3819324241915d917.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=308b501fbe8f4bf0be4684e1a098cf6a in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_8604: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_8604
override True
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

## Sync to high side

In [10]:
# syft absolute
from syft.client.syncing import compare_states

low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state = compare_states(low_state, high_state)

In [11]:
diff_state.diffs

compute_mean
test_l
Node: test_l, Status: pending
default-pool


[syft.service.sync.diff_state.Diff, syft.service.sync.diff_state.Diff]

In [12]:
# syft absolute
# state.objs_to_sync
from syft.client.syncing import resolve

low_items_to_sync, high_items_to_sync = resolve(diff_state, force_approve=True)

```python
class Request:
  id: str = 3a6a65dca0eb4ada8076453ee440a518
  request_time: str = 2024-02-22 18:24:00
  updated_at: str = None
  status: str = RequestStatus.PENDING
  changes: str = ['Request to change <b>compute_mean</b> (Pool Id: <b>default-pool</b>) to permission <b>RequestStatus.APPROVED</b>. No nested requests']
  requesting_user_verify_key: str = adf8a3a2dfa1917d7994eee6140893b02d2b8272ff462a02d03008164795d22d

```

```python
class UserCode
    id: UID = bf66fcbeaed34d0891fa3f8fa0620cb7
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

```

- What do we do with properties here?
  - just set them to None

In [13]:
len(low_items_to_sync), len(high_items_to_sync)

(0, 2)

In [14]:
client_low.apply_state(low_items_to_sync)

SyftSuccess: Synced 0 items

In [15]:
client_high.apply_state(high_items_to_sync)

SyftSuccess: Synced 2 items

# Run code high and sync back result

In [16]:
data_high = client_high.datasets[0].assets[0]

In [17]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

SyftInfo: Node Landed!


```python
class Job:
    id: UID = 5d767b6ac2674fcc97d0cea5b16c0664
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [18]:
# wait for the result
job_high.wait().get()

override True


17.0

In [19]:
job_info = job_high.info(public_metadata=True, result=True)

In [20]:
job_info.result

```python
Pointer
```
17.0

In [21]:
request = client_high.requests[0]
result_obj = job_high.result

print("result obj", result_obj.id)
accept_res = request.accept_by_depositing_result(job_info)

# request.approve()
# TODO: FIX THIS
request = client_high.requests[0]
code = request.code
log = job_high._get_log_objs()

result obj f36bce2300f24ea5a24bc8454f934ffa
Approving request for domain test_h
Approving request for domain test_h
ADDING PERMISSION [READ: f36bce2300f24ea5a24bc8454f934ffa as adf8a3a2dfa1917d7994eee6140893b02d2b8272ff462a02d03008164795d22d] f36bce2300f24ea5a24bc8454f934ffa
returning existing job
setting permission
None
None


In [22]:
action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [23]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state_2 = compare_states(low_state, high_state)

diff_state_2.diffs


2024-02-22 18:24:02.363962


[syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff]

In [24]:
high_state.rows

[syft.service.sync.sync_state.SyncStateRow,
 syft.service.sync.sync_state.SyncStateRow,
 syft.service.sync.sync_state.SyncStateRow,
 syft.service.sync.sync_state.SyncStateRow,
 syft.service.sync.sync_state.SyncStateRow]

In [25]:
low_items_to_sync, high_items_to_sync = resolve(diff_state_2, force_approve=True)

```python
class Job:
    id: UID = 5d767b6ac2674fcc97d0cea5b16c0664
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

```python
Pointer
```
17.0

```python
class SyftLog:
  id: str = 47fd70e6a34f420fb8527be0cb7e3954

```

In [26]:
low_items_to_sync

[syft.service.job.job_stash.Job,
 Pointer:
 17.0,
 syft.service.log.log.SyftLog,
 syft.service.request.request.Request,
 syft.service.code.user_code.UserCode]

In [27]:
client_low.apply_state(low_items_to_sync)

SyftSuccess: Synced 5 items

In [28]:
client_high.apply_state(high_items_to_sync)

SyftSuccess: Synced 0 items

In [29]:
# sync_items(low_items_to_sync, node_low, node_high, client_low)

In [30]:
# items_to_sync = [code, job_high.result, job_high, log]

In [31]:
# sync_items(items_to_sync, node_low, node_high, client_low)

In [32]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [33]:
res_low = client_low_ds.code.compute_mean(data=data_low)

override False


In [34]:
res_low.get()

17.0

In [35]:
assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_policy.last_output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [36]:
private_high.mean()

17.0

In [37]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)

bf66fcbeaed34d0891fa3f8fa0620cb7


SyftWarning: There are existing jobs for this user code, returning the latest one

In [38]:
job_low.wait().get()

17.0

In [39]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)